In [ ]:
import torch
import torch.nn as nn
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
import torchtext.transforms as T
from torch.hub import load_state_dict_from_url

padding_idx = 1
bos_idx = 0
eos_idx = 2
max_seq_len = 256
xlmr_vocab_path = r"https://download.pytorch.org/models/text/xlmr.vocab.pt"
xlmr_spm_model_path = r"https://download.pytorch.org/models/text/xlmr.sentencepiece.bpe.model"

text_transform = T.Sequential(
    T.SentencePieceTokenizer(xlmr_spm_model_path),
    T.VocabTransform(load_state_dict_from_url(xlmr_vocab_path)),
    T.Truncate(max_seq_len - 2),
    T.AddToken(token=bos_idx, begin=True),
    T.AddToken(token=eos_idx, begin=False),
)

In [ ]:
text_transform = XLMR_BASE_ENCODER.transform()

In [ ]:
pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 13.3 MB/s 
     |████████████████████████████████| 139 kB 44.8 MB/s 
     |████████████████████████████████| 776.3 MB 11 kB/s 
     |████████████████████████████████| 127 kB 62.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.0+cu113 requires torch==1.12.0, but you have torch 1.12.1 which is incompatible.
torchtext 0.13.0 requires torch==1.12.0, but you have torch 1.12.1 w

In [ ]:
from torchtext.datasets import SST2
from torch.utils.data import DataLoader
batch_size = 16

train_datapipe = SST2(split='train')
dev_datapipe = SST2(split='dev')

# Transform the raw dataset using non-batched API (i.e apply transformation line by line)
train_datapipe = train_datapipe.map(lambda x: (text_transform(x[0]), x[1]))
train_datapipe = train_datapipe.batch(batch_size)
train_datapipe = train_datapipe.rows2columnar(["token_ids", "target"])
train_dataloader = DataLoader(train_datapipe, batch_size=None)

dev_datapipe = dev_datapipe.map(lambda x: (text_transform(x[0]), x[1]))
dev_datapipe = dev_datapipe.batch(batch_size)
dev_datapipe = dev_datapipe.rows2columnar(["token_ids", "target"])
dev_dataloader = DataLoader(dev_datapipe, batch_size=None)

In [ ]:
from torchtext.datasets import SST2
from torch.utils.data import DataLoader
batch_size = 16

In [ ]:
train_datapipe = SST2(split='train')

In [ ]:
train_datapipe

ShardingFilterIterDataPipe

In [ ]:
num_classes = 2
input_dim = 768

from torchtext.models import RobertaClassificationHead, XLMR_BASE_ENCODER
classifier_head = RobertaClassificationHead(num_classes=num_classes, input_dim=input_dim)
model = XLMR_BASE_ENCODER.get_model(head=classifier_head)
model.to(DEVICE)

RobertaModel(
  (encoder): RobertaEncoder(
    (transformer): TransformerEncoder(
      (token_embedding): Embedding(250002, 768, padding_idx=1)
      (layers): TransformerEncoder(
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
            )
            (linear1): Linear(in_features=768, out_features=3072, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (linear2): Linear(in_features=3072, out_features=768, bias=True)
            (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout1): Dropout(p=0.1, inplace=False)
            (dropout2): Dropout(p=0.1, inplace=False)
          )
          (1): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (out_proj)

In [ ]:
import torchtext.functional as F
from torch.optim import AdamW

learning_rate = 1e-5
optim = AdamW(model.parameters(), lr=learning_rate)
criteria = nn.CrossEntropyLoss()


def train_step(input, target):
    output = model(input)
    loss = criteria(output, target)
    optim.zero_grad()
    loss.backward()
    optim.step()


def eval_step(input, target):
    output = model(input)
    loss = criteria(output, target).item()
    return float(loss), (output.argmax(1) == target).type(torch.float).sum().item()


def evaluate():
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    counter = 0
    with torch.no_grad():
        for batch in dev_dataloader:
            input = F.to_tensor(batch['token_ids'], padding_value=padding_idx).to(DEVICE)
            target = torch.tensor(batch['target']).to(DEVICE)
            loss, predictions = eval_step(input, target)
            total_loss += loss
            correct_predictions += predictions
            total_predictions += len(target)
            counter += 1

    return total_loss / counter, correct_predictions / total_predictions

In [ ]:
num_epochs = 5

for e in range(num_epochs):
    for batch in train_dataloader:
        input = F.to_tensor(batch['token_ids'], padding_value=padding_idx).to(DEVICE)
        target = torch.tensor(batch['target']).to(DEVICE)
        train_step(input, target)

    loss, accuracy = evaluate()
    print("Epoch = [{}], loss = [{}], accuracy = [{}]".format(e, loss, accuracy))

KeyboardInterrupt: ignored

NEXT

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 35.6 MB/s 
     |████████████████████████████████| 101 kB 9.4 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install wandb

In [ ]:
import pandas as pd
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os

In [ ]:
Corpus = pd.read_csv("/content/tagged02-07-2022-122843.csv",encoding='utf8')

In [ ]:
Corpus['Tag'] = [int(str(entry).replace('2','0')) for entry in Corpus['Tag']]
Corpus['Tag'] = [int(str(entry).replace('4','0')) for entry in Corpus['Tag']]
Corpus['Tag'] = [int(str(entry).replace('3','1')) for entry in Corpus['Tag']]

In [ ]:
# Step - a : Remove blank rows if any.
Corpus['dcNoteFinal'].dropna(inplace=True)

In [ ]:
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['dcNoteFinal'] = [entry.lower().replace('b"','') for entry in Corpus['dcNoteFinal']]

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Step - b2: remove punctuation

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
Corpus['dcNoteFinal'] = [tokenizer.tokenize(entry) for entry in Corpus['dcNoteFinal']]
#tokenizer.tokenize('Eighty-seven miles to go, yet.  Onward!')


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [ ]:
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['dcNoteFinal']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['Tag'],test_size=0.3)

In [ ]:
Train_X

130    ['emory', 'university', 'hospital', 'midtown',...
121    ['patient', 'jones', 'william', 'henry', 'mrn'...
96     ['emory', 'university', 'hospital', 'midtown',...
105    ['patient', 'skinner', 'arthur', 'james', 'mrn...
37     ['patient', 'smith', 'timothy', 'mrn', 'fin', ...
                             ...                        
94     ['emory', 'university', 'hospital', 'summary',...
64     ['patient', 'perry', 'helen', 'v', 'mrn', 'fin...
84     ['present', 'history', 'm', 'carr', 'yo', 'fem...
139    ['emory', 'university', 'hospital', 'discharge...
22     ['emory', 'university', 'hospital', 'midtown',...
Name: text_final, Length: 105, dtype: object

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 14.6 MB/s 
     |████████████████████████████████| 141 kB 58.4 MB/s 
     |████████████████████████████████| 212 kB 49.8 MB/s 
     |████████████████████████████████| 115 kB 54.0 MB/s 


In [ ]:
import datasets
train_data, test_data = datasets.load_dataset('imdb', split =['train', 'test'])

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
train_data

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = True, truncation=True)


train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data))
test_data = test_data.map(tokenization, batched = True, batch_size = len(test_data))

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
train_data

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [ ]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
train_data

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [ ]:
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

NEXT

In [ ]:
import os
import numpy as np
import pandas as pd
import transformers
import torch
from torch.utils.data import (
    Dataset,
    DataLoader,
    RandomSampler,
    SequentialSampler
)

import math
from transformers import  (
    BertPreTrainedModel,
    RobertaConfig,
    RobertaTokenizerFast
)

from transformers.optimization import (
    AdamW,
    get_linear_schedule_with_warmup
)

from scipy.special import softmax
from torch.nn import CrossEntropyLoss

from sklearn.metrics import (
    confusion_matrix,
    matthews_corrcoef,
    roc_curve,
    auc,
    average_precision_score,
)

from transformers.models.roberta.modeling_roberta import (
    RobertaClassificationHead,
    RobertaConfig,
    RobertaModel,
)

In [ ]:
model_name = 'pchanda/pretrained-smiles-pubchem10m'
num_labels = 2
device = torch.device("cuda")

tokenizer_name = model_name

max_seq_length = 128
train_batch_size = 8
test_batch_size = 8
warmup_ratio = 0.06
weight_decay=0.0
gradient_accumulation_steps = 1
num_train_epochs = 25
learning_rate = 1e-05
adam_epsilon = 1e-08

In [ ]:
class RobertaForSmilesClassification(BertPreTrainedModel):

    def __init__(self, config):
        super(RobertaForSmilesClassification, self).__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config)
        self.classifier = RobertaClassificationHead(config)


    def forward(self, input_ids, attention_mask, labels):
        outputs = self.roberta(input_ids,attention_mask=attention_mask)
        sequence_output = outputs[0]
        logits = self.classifier(sequence_output)

        outputs = (logits,) + outputs[2:]

        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [ ]:
config_class = RobertaConfig
model_class = RobertaForSmilesClassification
tokenizer_class = RobertaTokenizerFast

config = config_class.from_pretrained(model_name, num_labels=num_labels)

model = model_class.from_pretrained(model_name, config=config)
print('Model=\n',model,'\n')

tokenizer = tokenizer_class.from_pretrained(tokenizer_name, do_lower_case=False)
print('Tokenizer=',tokenizer,'\n')

In [ ]:
Corpus

,order,index,ENCNTR_ID,PERSON_ID,dcNoteFinal,Tag
0,0,0,63659760,88639569,"* patient: board, catherine ...",1
1,1,1,63927816,78432102,emory university hospital midtown summary name...,0
2,2,2,64686569,89329608,emory university hospital summary name: ...,0
3,3,3,65001068,69612068,* tx_rtf32 10.1.323.501* d469a1f3* emory unive...,0
4,4,4,65330296,90343247,emory university hospital summary name: ...,0
...,...,...,...,...,...,...
146,146,303,66866343,19337092,"* patient: wilmont, selena r ...",1
147,147,304,66767408,51120121,"* patient: moore, luther lavelle ...",1
148,148,310,66940878,90116249,* tx_rtf32 10.1.323.501* d469a1f3* emory unive...,1
149,149,311,66738084,18912196,emory university hospital midtown summary name...,1


In [ ]:
corpus_df = pd.DataFrame(Corpus,columns=['dcNoteFinal','Tag'])

In [ ]:
corpus_df

,dcNoteFinal,Tag
0,"* patient: board, catherine ...",1
1,emory university hospital midtown summary name...,0
2,emory university hospital summary name: ...,0
3,* tx_rtf32 10.1.323.501* d469a1f3* emory unive...,0
4,emory university hospital summary name: ...,0
...,...,...
146,"* patient: wilmont, selena r ...",1
147,"* patient: moore, luther lavelle ...",1
148,* tx_rtf32 10.1.323.501* d469a1f3* emory unive...,1
149,emory university hospital midtown summary name...,1


In [ ]:
train_df, test_df = model_selection.train_test_split(corpus_df,test_size=0.3)

In [ ]:
test_df

,dcNoteFinal,Tag
77,"*hammond, joshua t: 01/18/2013 17:07connor, mi...",0
128,emory university hospital summary name: ...,1
133,"* patient: hanks, tammy navette ham ...",1
69,* tx_rtf32 10.1.323.501* d469a1f3** d469a1f3**...,0
96,* tx_rtf32 10.1.323.501* d469a1f3* emory unive...,1
58,emory university hospital midtown summary name...,0
3,* tx_rtf32 10.1.323.501* d469a1f3* emory unive...,0
19,emory university hospital midtown summary name...,0
46,"* patient: lamarr, anna louise ...",0
116,emory university hospital midtown summary name...,0


In [ ]:
train_df['text_final'] = (" ".join(entry) for entry in train_df['text_final'])
test_df['text_final'] = (" ".join(entry) for entry in test_df['text_final'])

TypeError: ignored

In [ ]:
for entry in train_df['text_final']:
  print(entry.tolist())
  #print(" ".join(entry))

In [ ]:
url = 'https://raw.githubusercontent.com/pchanda/pchanda.github.io/master/data/bertMol/train_data.txt'
train_df = pd.read_csv(url,delimiter='\t')
print('Training data...')
print (train_df.head())

url = 'https://raw.githubusercontent.com/pchanda/pchanda.github.io/master/data/bertMol/test_data.txt'
test_df = pd.read_csv(url,delimiter='\t')
print('\nTesting data...')
print (test_df.head())

Training data...
                                                text  label
0                 COc1cc(nc(OC)n1)c2cn(nc2C)c3cccnc3      0
1     CCN(C(=O)NC(=O)c1ccc(F)cc1)c2cn(nc2Cl)c3cccnc3      0
2             CCc1noc(n1)C2CCN(CC2)C(=O)N(C)c3cccnc3      0
3               CCN(C(=O)C(C)CBr)c1sc(nc1Cl)c2cccnc2      0
4  CCS(=NC(=O)c1cc(Cl)cc(C)c1NC(=O)c2cnc(nc2c3ncc...      0

Testing data...
                                                text  label
0             CN(C(=O)CCC(F)(F)F)c1cn(nc1Br)c2cccnc2      0
1  COC1CCC2(CC1)NC(=O)C(=C2[O-])c3c(C)cc(cc3C)n4c...      0
2        CSCCN(C(=O)OC(C)(C)C)c1sc([nH+]c1C)c2cccnc2      0
3                     CCOC(=O)c1cn2nc(sc2n1)c3cccnc3      0
4                      CSCCC(=O)Nc1sc(nc1Cl)c2cccnc2      0


In [ ]:
class MyClassificationDataset(Dataset):

    def __init__(self, data, tokenizer):
        text, labels = data
        self.examples = tokenizer(text=text,text_pair=None,truncation=True,padding="max_length",
                                  max_length=max_seq_length,return_tensors="pt")
        self.labels = torch.tensor(labels, dtype=torch.long)


    def __len__(self):
        return len(self.examples["input_ids"])

    def __getitem__(self, index):
        return {key: self.examples[key][index] for key in self.examples}, self.labels[index]


train_examples = (train_df.iloc[:, 0].astype(str).tolist(), train_df.iloc[:, 1].tolist())
train_dataset = MyClassificationDataset(train_examples,tokenizer)

test_examples = (test_df.iloc[:, 0].astype(str).tolist(), test_df.iloc[:, 1].tolist())
test_dataset = MyClassificationDataset(test_examples,tokenizer)

TypeError: ignored

In [ ]:
train_examples = (train_df.iloc[:, 0].astype(str).tolist(), train_df.iloc[:, 1].tolist())

In [ ]:
train_examples

(['emory university hospital midtown summary name:                        clay, richard n:                                 1426328 no:                        8807412347 date:                          12/13/2012 date:                      12/14/2012 no:                               050156 md:                        chandanreddy devireddy, m.d. list:. symptomatic carotid artery disease, reason for initial hospitalization and    procedure.    a.     status post carotid angiogram on 12/13/2012 after evidence of high-     grade carotid stenosis on duplex ultrasound on the left ica.    b.     carotid angiogram revealed a 40% right ica stenosis and an 80% left     ica stenosis, and cerebral runoff reveals chronic occluded left anterior     cerebral artery.    c.     status post successful carotid artery stenting with embolic     protection at the right internal carotid artery with improvement in     stenosis from 80% to approximately 15% residual stenosis and received a     protege 10 to 7 x

In [ ]:
def get_inputs_dict(batch):
    inputs = {key: value.squeeze(1).to(device) for key, value in batch[0].items()}
    inputs["labels"] = batch[1].to(device)
    return inputs

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset,sampler=train_sampler,batch_size=train_batch_size)

test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=test_batch_size)

#Extract a batch as sanity-check
batch = get_inputs_dict(next(iter(train_dataloader)))
input_ids = batch['input_ids'].to(device)
attention_mask = batch['attention_mask'].to(device)
labels = batch['labels'].to(device)

print(batch)